In [77]:
import requests
import pandas as pd
import numpy as np

In [41]:
def divide_chunks(n, d=100):
    return int((n+ (d-1))/d)

In [42]:
base_url = "https://api.fabdb.net/cards"

payload = {
    "page": 1,
    "per_page": 100,
}

pages = []
for i in range(1, divide_chunks(656, 100)):
    r = requests.get(base_url, params=payload)
    pages.append(r.json())
    payload['page'] += 1

In [65]:
dfs = []
for i in pages:
    dfs.append(pd.DataFrame.from_dict(i['data']))
df = pd.concat(dfs).reset_index()

In [67]:
stat_list = df.stats.values.tolist()
for n, i in enumerate(stat_list):
    if i == []:
        stat_list[n] = {}
        
df = df.drop(['stats', 'index'], axis=1).join(pd.DataFrame(stat_list))

In [84]:
def transform_name(row):

    resource_map = {
        '3': '(Blue)',
        '2': '(Yellow)',
        '1': '(Red)',
        np.nan: '',
    }
    
    return f"{row['name']} {resource_map[row['resource']]}"

In [85]:
df['name_resource'] = df.apply(transform_name, axis=1)

In [88]:
df.to_csv("cards.csv", index=False)

In [86]:
df.head()

,identifier,image,keywords,name,rarity,text,variants,attack,cost,defense,intellect,life,resource,name_resource
0,ARC000,https://fabdb2.imgix.net/cards/arc/0.png?w=300...,"[generic, resource, gem]",Eye of Ophidia,F,**Legendary** *(You may only have 1 Eye of Oph...,[],NaN,NaN,NaN,NaN,NaN,3,Eye of Ophidia (Blue)
1,ARC001,https://fabdb2.imgix.net/cards/arc/1.png?w=300...,"[mechanologist, hero]","Dash, Inventor Extraordinaire",T,You may start the game with a Mechanologist it...,[],NaN,NaN,NaN,4,40,NaN,"Dash, Inventor Extraordinaire"
2,ARC002,https://fabdb2.imgix.net/cards/arc/2.png?w=300...,"[mechanologist, hero, young]",Dash,T,You may start the game with a Mechanologist it...,[],NaN,NaN,NaN,4,20,NaN,Dash
3,ARC003,https://fabdb2.imgix.net/cards/arc/3.png?w=300...,"[mechanologist, weapon, pistol, 2h]",Teklo Plasma Pistol,T,**Action** - Remove a steam counter from Teklo...,[],2,NaN,NaN,NaN,NaN,NaN,Teklo Plasma Pistol
4,ARC004,https://fabdb2.imgix.net/cards/arc/4.png?w=300...,"[mechanologist, equipment, chest]",Teklo Foundry Heart,L,**Once per Turn Action** - [resource]: Banish ...,[],NaN,NaN,2,NaN,NaN,NaN,Teklo Foundry Heart


In [70]:
df[df['name'] == "Command and Conquer"]

,identifier,image,keywords,name,rarity,text,variants,attack,cost,defense,intellect,life,resource
159,ARC159,https://fabdb2.imgix.net/cards/arc/159.png?w=3...,"[generic, action, attack]",Command and Conquer,M,Defense reactions can't be played to Command a...,[],6,2,3,NaN,NaN,1
